In [1]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.tools.tool_context import ToolContext
from google.adk.sessions import InMemorySessionService

In [2]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [3]:
def save_info(tool_context: ToolContext, food: str, emotion: str):
    """
    Tool to record and save user food preferences in session state.

    Args:
        food: The food preference to store in session state
        emotion: The emotion associated with the food preference
    """
    tool_context.state["user:food"] = (tool_context.state.get("user:food") or {}) | {food: emotion}
    return {"status": "success"}


def retrieve_info(tool_context: ToolContext):
    """
    Tool to retrieve user food preferences from session state.
    """
    food_preferences = tool_context.state.get("user:food") or {}
    return {"status": "success", "food_preferences": "".join(f"{k}: {v}, " for k, v in food_preferences.items())}

In [4]:
assistant_agent = Agent(
    name="AssistantAgent",
    model=model,
    instruction="""
    You are a personal assistant. Help me with all my tasks.
    Additionally, to manage user context preferences:
    * To save food preferences when provided use `save_info` tool. 
    * To fetch user food preferences when required use `retrieve_info` tool.
    """,
    tools=[save_info, retrieve_info],
)

In [5]:
session_service = InMemorySessionService()

In [6]:
runner = Runner(agent=assistant_agent, app_name="Runner", session_service=session_service)

In [7]:
_ = await runner.run_debug("I love eating ice cream. What are some other desserts I might like?")


 ### Created new session: debug_session_id

User > I love eating ice cream. What are some other desserts I might like?


AssistantAgent > I've noted that you love ice cream! That's a great start. Since you enjoy ice cream, you might also like other frozen desserts such as gelato or sorbet. Cakes, pies, and cookies are also popular choices that many ice cream lovers enjoy.

Would you like me to help you find recipes for any of these?



In [8]:
_ = await runner.run_debug("What do you know about my food preferences?")


 ### Continue session: debug_session_id

User > What do you know about my food preferences?
AssistantAgent > I know that you love ice cream! Is there anything else you'd like to share or ask about?
